In [1]:
include("../lib/Lib.jl")
include("../lib/DeepSpace.jl")

Main.DeepSpace

In [2]:
using DataFrames, DataFramesMeta
using CSV
using Statistics

In [3]:
KEY = "2019chs_wk1"
YEAR = "2019"
COLUMN = :teleopSwitchOwnershipSec

:teleopSwitchOwnershipSec

In [4]:
(headers, scores, teams) = Lib.read_stacked(KEY, YEAR)
scores[:color] = repeat(["r", "b"], size(scores,1) >> 1)
teams[:color] = repeat(["r", "b"], size(teams,1) >> 1)

360-element Array{String,1}:
 "r"
 "b"
 "r"
 "b"
 "r"
 "b"
 "r"
 "b"
 "r"
 "b"
 "r"
 "b"
 "r"
 ⋮  
 "r"
 "b"
 "r"
 "b"
 "r"
 "b"
 "r"
 "b"
 "r"
 "b"
 "r"
 "b"

In [5]:
DeepSpace.calculated_columns!(scores)
scores[:ownPoints] = scores.totalPoints .- scores.foulPoints .- (3 .* scores.foulCount) .- (9 * scores.techFoulCount)

360-element Array{Int64,1}:
  62
  82
  72
  71
  66
  65
  52
  54
  62
  35
  59
 -25
  63
   ⋮
  20
  12
  70
  45
  59
  47
  57
  60
  74
  66
  51
  55

In [6]:
scores_red = @linq scores |> where(:color .== "r")
scores_blue = @linq scores |> where(:color .== "b")
teams_red = @linq teams |> where(:color .== "r") |> select(:team1, :team2, :team3, :key)
teams_blue = @linq teams |> where(:color .== "b") |> select(:team1, :team2, :team3, :key)

,team1,team2,team3,key
,String⍰,String⍰,String⍰,String⍰
1,frc1418,frc614,frc6543,2019vahay_f1m1_blue
2,frc1418,frc614,frc6543,2019vahay_f1m2_blue
3,frc1418,frc614,frc6543,2019vahay_f1m3_blue
4,frc2988,frc116,frc611,2019vahay_qf1m1_blue
5,frc2988,frc116,frc611,2019vahay_qf1m2_blue
6,frc2988,frc116,frc611,2019vahay_qf1m3_blue
7,frc4242,frc339,frc2186,2019vahay_qf2m1_blue
8,frc4242,frc339,frc2186,2019vahay_qf2m2_blue
9,frc888,frc2028,frc3455,2019vahay_qf3m1_blue


In [7]:
scoreline(teams, scores) = let stacked = stack(teams, [:team1, :team2, :team3], variable_name=:role, value_name=:team)
    join(scores, stacked, on=:key)
end

scoreline_blue = scoreline(teams_blue, scores_blue)
scoreline_red = scoreline(teams_red, scores_red)

,adjustPoints,autoPoints,bay1,bay2,bay3,bay4,bay5,bay6,bay7,bay8,cargoPoints,completeRocketRankingPoint,completedRocketFar,completedRocketNear,endgameRobot1,endgameRobot2,endgameRobot3,foulCount,foulPoints,habClimbPoints,habDockingRankingPoint,habLineRobot1,habLineRobot2,habLineRobot3,hatchPanelPoints,lowLeftRocketFar,lowLeftRocketNear,lowRightRocketFar,lowRightRocketNear,midLeftRocketFar,midLeftRocketNear,midRightRocketFar,midRightRocketNear,preMatchBay1,preMatchBay2,preMatchBay3,preMatchBay6,preMatchBay7,preMatchBay8,preMatchLevelRobot1,preMatchLevelRobot2,preMatchLevelRobot3,rp,sandStormBonusPoints,techFoulCount,teleopPoints,topLeftRocketFar,topLeftRocketNear,topRightRocketFar,topRightRocketNear,totalPoints,key,level,event,color,rocketLowPoints,rocketMidPoints,rocketTopPoints,rocketPoints,ownPoints,role,team
,Int64⍰,Int64⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,Int64⍰,Bool⍰,Bool⍰,Bool⍰,String⍰,String⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Bool⍰,String⍰,String⍰,String⍰,Int64⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,String⍰,String⍰,String⍰,String⍰,Int64⍰,String⍰,String⍰,String⍰,String,Int64,Int64,Int64,Int64,Int64,Symbol,String⍰
1,0,15,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,Panel,33,false,false,false,HabLevel1,HabLevel2,None,0,3,9,false,CrossedHabLineInSandstorm,CrossedHabLineInSandstorm,CrossedHabLineInSandstorm,14,PanelAndCargo,Panel,PanelAndCargo,None,None,PanelAndCargo,None,PanelAndCargo,Panel,Panel,Panel,Panel,Panel,Panel,HabLevel2,HabLevel2,HabLevel1,0,15,1,56,None,None,None,None,74,2019vahay_f1m1_red,f,2019vahay,r,12,10,0,22,62,team1,frc2363
2,0,15,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,Panel,33,false,false,false,HabLevel1,HabLevel2,None,0,3,9,false,CrossedHabLineInSandstorm,CrossedHabLineInSandstorm,CrossedHabLineInSandstorm,14,PanelAndCargo,Panel,PanelAndCargo,None,None,PanelAndCargo,None,PanelAndCargo,Panel,Panel,Panel,Panel,Panel,Panel,HabLevel2,HabLevel2,HabLevel1,0,15,1,56,None,None,None,None,74,2019vahay_f1m1_red,f,2019vahay,r,12,10,0,22,62,team2,frc612
3,0,15,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,Panel,33,false,false,false,HabLevel1,HabLevel2,None,0,3,9,false,CrossedHabLineInSandstorm,CrossedHabLineInSandstorm,CrossedHabLineInSandstorm,14,PanelAndCargo,Panel,PanelAndCargo,None,None,PanelAndCargo,None,PanelAndCargo,Panel,Panel,Panel,Panel,Panel,Panel,HabLevel2,HabLevel2,HabLevel1,0,15,1,56,None,None,None,None,74,2019vahay_f1m1_red,f,2019vahay,r,12,10,0,22,62,team3,frc1731
4,0,15,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,Panel,30,false,false,false,HabLevel1,HabLevel1,HabLevel1,0,0,9,false,CrossedHabLineInSandstorm,CrossedHabLineInSandstorm,CrossedHabLineInSandstorm,18,Panel,Panel,PanelAndCargo,Panel,None,PanelAndCargo,Panel,PanelAndCargo,Panel,Panel,Panel,Panel,Panel,Panel,HabLevel2,HabLevel2,HabLevel1,0,15,0,57,None,None,None,None,72,2019vahay_f1m2_red,f,2019vahay,r,11,12,0,23,72,team1,frc2363
5,0,15,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,Panel,30,false,false,false,HabLevel1,HabLevel1,HabLevel1,0,0,9,false,CrossedHabLineInSandstorm,CrossedHabLineInSandstorm,CrossedHabLineInSandstorm,18,Panel,Panel,PanelAndCargo,Panel,None,PanelAndCargo,Panel,PanelAndCargo,Panel,Panel,Panel,Panel,Panel,Panel,HabLevel2,HabLevel2,HabLevel1,0,15,0,57,None,None,None,None,72,2019vahay_f1m2_red,f,2019vahay,r,11,12,0,23,72,team2,frc612
6,0,15,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,PanelAndCargo,Panel,30,false,false,false,HabLevel1,HabLevel1,HabLevel1,0,0,9,false,CrossedHabLineInSandstorm,CrossedHabLineInSandstorm,CrossedHabLineInSandstorm,18,Panel,Panel,PanelAndCargo,

In [8]:
numericcols = Lib.numeric_col_names(scoreline_red)
diff_red = convert(Matrix,scoreline_red[:,numericcols]) .- convert(Matrix, scoreline_blue[:,numericcols])
diff_blue = convert(Matrix,scoreline_blue[:,numericcols]) .- convert(Matrix, scoreline_red[:,numericcols])

diffdf = DataFrame(vcat(diff_red, diff_blue), numericcols)
diffdf[:key] = vcat(scoreline_red.key, scoreline_blue.key)
diffdf[:team] = vcat(scoreline_red.team, scoreline_blue.team)

1080-element Array{Union{Missing, String},1}:
 "frc2363"
 "frc612" 
 "frc1731"
 "frc2363"
 "frc612" 
 "frc1731"
 "frc2363"
 "frc612" 
 "frc1731"
 "frc2363"
 "frc612" 
 "frc1731"
 "frc2363"
 ⋮        
 "frc2106"
 "frc1599"
 "frc5804"
 "frc1610"
 "frc384" 
 "frc3359"
 "frc1610"
 "frc384" 
 "frc3359"
 "frc1610"
 "frc384" 
 "frc3359"

In [9]:
win(pointsMargin) = 
    if pointsMargin > 0 
        1.0
    elseif pointsMargin < 0 
        0.0
    else 
        0.5
    end

diffdf[:win] = win.(diffdf.totalPoints)
push!(numericcols, :win)

22-element Array{Symbol,1}:
 :adjustPoints              
 :autoPoints                
 :cargoPoints               
 :completeRocketRankingPoint
 :completedRocketFar        
 :completedRocketNear       
 :foulCount                 
 :foulPoints                
 :habClimbPoints            
 :habDockingRankingPoint    
 :hatchPanelPoints          
 :rp                        
 :sandStormBonusPoints      
 :techFoulCount             
 :teleopPoints              
 :totalPoints               
 :rocketLowPoints           
 :rocketMidPoints           
 :rocketTopPoints           
 :rocketPoints              
 :ownPoints                 
 :win                       

In [10]:
plusminus = by(diffdf, :team, Dict(n => mean for n in numericcols)...)

,team,adjustPoints_mean,foulPoints_mean,cargoPoints_mean,completeRocketRankingPoint_mean,rocketPoints_mean,rp_mean,sandStormBonusPoints_mean,rocketLowPoints_mean,rocketMidPoints_mean,teleopPoints_mean,win_mean,autoPoints_mean,completedRocketNear_mean,habDockingRankingPoint_mean,completedRocketFar_mean,totalPoints_mean,hatchPanelPoints_mean,foulCount_mean,habClimbPoints_mean,ownPoints_mean,rocketTopPoints_mean,techFoulCount_mean
,String⍰,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,frc2363,3.2,0.7,0.9,0.0,3.6,0.55,2.4,-0.4,3.9,5.35,0.65,2.4,0.0,0.25,0.0,11.65,2.2,-0.4,2.25,11.7,0.1,0.05
2,frc612,3.2,1.05,1.95,0.0,2.7,1.5,2.55,0.3,2.8,8.4,0.85,2.55,0.0,0.4,0.0,15.2,2.4,-0.35,4.05,15.2,-0.4,0.0
3,frc1731,3.04762,0.857143,1.0,0.0,3.90476,-0.238095,1.71429,1.33333,2.47619,0.619048,0.52381,1.71429,0.0,-0.047619,0.0,6.2381,1.61905,-0.285714,-2.0,6.2381,0.0952381,0.0
4,frc4099,0.0,1.1875,3.5625,0.0,2.6875,0.75,1.6875,2.8125,-0.6875,8.5,0.6875,1.6875,0.0,0.125,0.0,11.375,1.75,-0.1875,3.1875,11.3125,0.5625,-0.0625
5,frc5338,0.0,1.9375,0.9375,0.0,-4.25,0.875,0.0,-1.0625,-2.75,5.5,0.625,0.0,0.0,0.5,0.0,7.4375,-1.25,-0.4375,5.8125,7.375,-0.4375,-0.0625
6,frc4505,0.0,-1.0,1.3125,0.0,-0.4375,-0.25,1.875,-0.375,-0.3125,0.8125,0.5,1.875,0.0,-0.125,0.0,1.6875,-0.125,0.125,-0.375,1.75,0.25,0.0625
7,frc1418,0.0,0.7,1.8,0.05,11.05,0.4,-1.05,5.25,2.8,8.95,0.6,-1.05,0.0,0.05,0.0,8.6,6.4,0.1,0.75,8.5,3.0,-0.1
8,frc614,0.0,3.75,6.15,0.0,2.65,1.15,1.5,2.6,0.3,12.4,0.75,1.5,0.0,0.25,0.0,17.65,1.9,-0.75,4.35,17.5,-0.25,-0.15
9,frc6543,0.0,0.2,1.2,-0.05,0.55,-0.75,-1.35,1.35,-0.1,0.3,0.4,-1.35,0.0,-0.2,0.0,-0.85,0.3,0.6,-1.2,-1.05,-0.7,-0.2


In [11]:
showbest(col) = sort(plusminus, col, rev=true)[:,[:team, col]]
showworst(col) = sort(plusminus, col)[:,[:team, col]]

showbest(:win_mean)

,team,win_mean
,String⍰,Float64
1,frc346,0.916667
2,frc612,0.85
3,frc1262,0.833333
4,frc2534,0.764706
5,frc977,0.764706
6,frc614,0.75
7,frc1610,0.736842
8,frc339,0.692308
9,frc4099,0.6875


In [12]:
using CSV
CSV.write("../output/$(KEY)_plusminus.csv", plusminus)

"../output/2019chs_wk1_plusminus.csv"